In [45]:
import sys
import time
import math
import copy
import random
import string
import numpy as np
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tqdm import tqdm
from os import listdir

STEMMING = True
LOWER_CASE = True
porter_stemmer = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')
# nltk.download('stopwords')
bad_words = {'aed','oed','eed'} # these words fail in nltk stemmer algorithm

In [46]:
def loadDir(name,stemming,lower_case):
    # Loads the files in the folder and returns a list of lists of words from the text in each file
    X0 = []
    count = 0
    for f in tqdm(listdir(name)):
        fullname = name+f
        text = []
        with open(fullname, 'rb') as f:
            for line in f:
                if lower_case:
                    line = line.decode(errors='ignore').lower()
                    text += tokenizer.tokenize(line)
                else:
                    text += tokenizer.tokenize(line.decode(errors='ignore'))
        if stemming:
            for i in range(len(text)):
                if text[i] in bad_words:
                    continue
                text[i] = porter_stemmer.stem(text[i])
        X0.append(text)
        count = count + 1
    return X0

In [47]:
def loadDataset(directory, stemming, lower_case):
    positive = loadDir(directory + '/pos/',stemming, lower_case)
    negative = loadDir(directory + '/neg/',stemming, lower_case)
    random.shuffle(positive)
    boundaryTrain = math.floor(0.8 * len(positive))
    trainPos = positive[:boundaryTrain]
    random.shuffle(negative)
    trainNeg = negative[:boundaryTrain]
    combinedTrain = trainPos + trainNeg
    length = len(trainPos) + len(trainNeg)
    labelsTrain = len(trainNeg) * [1] + len(trainNeg) * [0]
    labelsTrain = np.array(labelsTrain)

    testPos = positive[boundaryTrain:]
    testNeg = negative[boundaryTrain:]
    combinedTest = testPos + testNeg
    labelsTest = len(testPos) * [1] + len(testNeg) * [0]
    labelsTest = np.array(labelsTest)
    return combinedTrain, labelsTrain, combinedTest, labelsTest

In [48]:

from nltk.corpus import stopwords
import time
import math

STEMMING = True
LOWER_CASE = True

def naiveBayes(train_set, train_labels, dev_set, smoothing_parameter, pos_prior):
    """
    train_set - List of list of words corresponding with each movie review
    example: suppose I had two reviews 'like this movie' and 'i fall asleep' in my training set
    Then train_set := [['like','this','movie'], ['i','fall','asleep']]

    train_labels - List of labels corresponding with train_set
    example: Suppose I had two reviews, first one was positive and second one was negative.
    Then train_labels := [1, 0]

    dev_set - List of list of words corresponding with each review that we are testing on
              It follows the same format as train_set

    smoothing_parameter - The smoothing parameter you provided with --laplace (1.0 by default)
    """
    # TODO: Write your code here
    # return predicted labels of development set
    # print("not even started yet")
    stops = stopwords.words('english') + list(string.punctuation)
    mydict = {}
    smoothing_parameter = 0.034
    posV = 0
    negV = 0
    totalposwords = 0
    totalnegwords = 0

    start = time.process_time()
    print("creating occurences and wordlist")

    #create bag of words and number of occurences
    count = 0
    for x in train_set:
        rating = train_labels[count]
        count += 1
        if(rating):
            for y in x:
                if y not in mydict and y not in stops:
                    mydict[y] = [1,0] #default [1 pos, 0 neg]
                    posV += 1
                    totalposwords += 1
                elif y not in stops:
                    if mydict[y][0] == 0:
                        posV += 1
                    mydict[y][0] += 1
                    totalposwords += 1
        else:
            for y in x:
                if y not in mydict and y not in stops:
                    mydict[y] = [0,1] #default [0 pos, 1 neg]
                    negV += 1
                    totalnegwords += 1
                elif y not in stops:
                    if mydict[y][1] == 0:
                        negV += 1
                    mydict[y][1] += 1
                    totalnegwords += 1
    print("review count is: ", count)
    print("posV", posV)
    print("negV", negV)
    print("total word count is:", totalposwords + totalnegwords)
    print("Going through train set took: ", time.process_time() - start)

    
    #come up with the bag of words unigram model
    probWordPos = {}
    probWordNeg = {}
    # PosII = 0
    # NegII = 0
    start = time.process_time()
    print("calculate prob")
    for x in mydict:
        #use laplace smoothing
        # count(W) + a / n + a * (V+1)
        # n = number of words in our UK training data
        # count(W) = number of times W appeared in UK training data
        # α is a tuning constant between 0 and 1 (typically small)
        # V = number of word TYPES seen in training data

        probWordPos[x] = math.log((mydict[x][0] + smoothing_parameter) / (totalposwords + smoothing_parameter * (posV + 1)))
        probWordNeg[x] = math.log((mydict[x][1] + smoothing_parameter) / (totalnegwords + smoothing_parameter * (negV + 1)))
        #calculate that II symbol that is basically summation, but using mutiplication
        #logs are used because we are working with incredibly small numbers
        # PosII += (probWordPos[x])
        # NegII += (probWordNeg[x])
    

    #unneccessary calculations on prob of train set
    print("Prob calculations: ", time.process_time() - start)
    # print("PosII", PosII)
    # print("NegII", NegII)
    # probPos = math.log(pos_prior) + PosII
    # probNeg = math.log(1 - pos_prior) + NegII

    # print("positive", probPos)
    # print("negative", probNeg)


    start = time.process_time()
    # #multiply by (add log) the pos prior, which is the other part of our equation in the unigram model
    # time to work with the dev set
    predictions = []
    for x in range(len(dev_set)):
        chancePos = math.log(pos_prior)
        chanceNeg = math.log(1-pos_prior)
        for y in range(len(dev_set[x])):
            if dev_set[x][y] in mydict:
                chancePos += probWordPos[dev_set[x][y]]
                chanceNeg += probWordNeg[dev_set[x][y]]
            # else:
                # chancePos += math.log((smoothing_parameter) / (totalposwords + smoothing_parameter * (posV + 1)))
                # chanceNeg += math.log((smoothing_parameter) / (totalnegwords + smoothing_parameter * (negV + 1)))
        if(chancePos > chanceNeg):
            predictions.append(1)
        else:
            predictions.append(0)
    print("devset time took:", time.process_time() - start)
    return predictions

In [49]:
######################### Mixed Model approach (unigram+bigram) ###################
# def naiveBayes(train_set, train_labels, dev_set, smoothing_parameter, pos_prior):
#     """
#     train_set - List of list of words corresponding with each movie review
#     example: suppose I had two reviews 'like this movie' and 'i fall asleep' in my training set
#     Then train_set := [['like','this','movie'], ['i','fall','asleep']]

#     train_labels - List of labels corresponding with train_set
#     example: Suppose I had two reviews, first one was positive and second one was negative.
#     Then train_labels := [1, 0]

#     dev_set - List of list of words corresponding with each review that we are testing on
#               It follows the same format as train_set

#     smoothing_parameter - The smoothing parameter you provided with --laplace (1.0 by default)
#     """
#     begin = time.process_time()
#     stops = stopwords.words('english') + list(string.punctuation)
#     mydict = {}
#     mybidict = {}
#     smoothing_parameter = 0.21
#     smoothing_parameter_bi = 0.75
#     posV = 0
#     posVbi = 0
#     negV = 0
#     posVbi = 0
#     negVbi = 0
#     totalposwords = 0
#     totalposwordsbi = 0
#     totalnegwords = 0
#     totalnegwordsbi = 0

#     print("creating occurences and wordlist")
#     #create bag of words and number of occurences

#     start = time.process_time()
#     count = 0
#     for x in train_set:
#         rating = train_labels[count]
#         count += 1
#         if(rating):
#             for y in x:
#                 if y in stops:
#                     continue
#                 if y not in mydict:
#                     mydict[y] = [1,0] #default [1 pos, 0 neg]
#                     posV += 1
#                     totalposwords += 1
#                 else:
#                     if mydict[y][0] == 0:
#                         posV += 1
#                     mydict[y][0] += 1
#                     totalposwords += 1
#             for y,z in zip(x, x[1:]):
#                 if y in stops or z in stops:
#                     continue
#                 if (y,z) not in mybidict:
#                     mybidict[(y,z)] = [1,0]
#                     posVbi += 1
#                     totalposwordsbi += 1
#                 else:
#                     if mybidict[(y,z)][0] == 0:
#                         posVbi += 1
#                     mybidict[(y,z)][0] += 1
#                     totalposwordsbi += 1
#                     # print ("[x,y] : ", [x,y])
#         else:
#             for y in x:
#                 if y in stops:
#                     continue
#                 if y not in mydict:
#                     mydict[y] = [0,1] #default [0 pos, 1 neg]
#                     negV += 1
#                     totalnegwords += 1
#                 else:
#                     if mydict[y][1] == 0:
#                         negV += 1
#                     mydict[y][1] += 1
#                     totalnegwords += 1
#             for y,z in zip(x, x[1:]):
#                 if y in stops or z in stops:
#                     continue
#                 if (y,z) not in mybidict:
#                     # print("(",y,",", z, ")")
#                     mybidict[(y,z)] = [0,1]
#                     negVbi += 1
#                     totalnegwordsbi += 1
#                 else:
#                     if mybidict[(y,z)][1] == 0:
#                         negVbi += 1
#                     mybidict[(y,z)][1] += 1
#                     totalnegwordsbi += 1
#     print("review count is: ", count)
#     print("posV", posV)
#     print("negV", negV)
#     print("total word count is:", totalposwords + totalnegwords)
#     print("posVbi", posVbi)
#     print("negVbi", negVbi)
#     print("total bi pair count is:", totalposwordsbi  + totalnegwordsbi)
#     print("Going through train took: ", time.process_time() - start)

    
#     #come up with the bag of words
#     probWordPos = {}
#     probWordNeg = {}
#     # PosII = 0
#     # NegII = 0
#     # print("calculate prob")
#     for x in mydict:
#         #use laplace smoothing
#         # count(W) + a / n + a * (V+1)
#         # n = number of words in our UK training data
#         # count(W) = number of times W appeared in UK training data
#         # α is a tuning constant between 0 and 1 (typically small)
#         # V = number of word TYPES seen in training data

#         probWordPos[x] = math.log((mydict[x][0] + smoothing_parameter) / (totalposwords + smoothing_parameter * (posV + 1)))
#         probWordNeg[x] = math.log((mydict[x][1] + smoothing_parameter) / (totalnegwords + smoothing_parameter * (negV + 1)))
#         #calculate that II symbol that is basically summation, but using mutiplication
#         #logs are used because we are working with incredibly small numbers
#         # PosII += (probWordPos[x])
#         # NegII += (probWordNeg[x])
#     # start = time.process_time()
#     probPairPos = {}
#     probPairNeg = {}
#     for x in mybidict:
#         probPairPos[x] = math.log((mybidict[x][0] + smoothing_parameter_bi) / (totalposwordsbi + smoothing_parameter_bi * (posVbi + 1)))
#         probPairNeg[x] = math.log((mybidict[x][1] + smoothing_parameter_bi) / (totalnegwordsbi + smoothing_parameter_bi * (negVbi + 1)))
#     #unneccessary calculations on prob of train set
#     # print("time to print bi dict", time.process_time() - start)
#     # print("PosII", PosII)
#     # print("NegII", NegII)
#     # probPos = math.log(pos_prior) + PosII
#     # probNeg = math.log(1 - pos_prior) + NegII

#     # print("positive", probPos)
#     # print("negative", probNeg)
    
#     start = time.process_time()
#     # #multiply by (add log) the pos prior, which is the other part of our equation in the unigram    model
#     # time to work with the dev set
#     predictions = []
#     lambd = 0.475
#     lambdaUni = lambd
#     lambdaBi = 1 - lambd
#     for x in range(len(dev_set)):
#         chancePosUni = math.log(pos_prior)
#         chanceNegUni = math.log(1-pos_prior)
#         chancePosBi = math.log(pos_prior)
#         chanceNegBi = math.log(1-pos_prior)
#         for y in range(len(dev_set[x])):
#             if dev_set[x][y] in mydict:
#                 chancePosUni += probWordPos[dev_set[x][y]]
#                 chanceNegUni += probWordNeg[dev_set[x][y]]
#             # else:
#                 # chancePos += math.log((smoothing_parameter) / (totalposwords + smoothing_parameter * (posV + 1)))
#                 # chanceNeg += math.log((smoothing_parameter) / (totalnegwords + smoothing_parameter * (negV + 1)))
#         for y,z in zip(dev_set[x], dev_set[x][1:]):
#             if (y,z) in mybidict:
#                 chancePosBi += probPairPos[(y,z)]
#                 chanceNegBi += probPairNeg[(y,z)]
#         chancePos = (lambdaBi * chancePosBi) + (lambdaUni * chancePosUni)
#         chanceNeg = (lambdaBi * chanceNegBi) + (lambdaUni * chanceNegUni)
#         if(chancePos > chanceNeg):
#             predictions.append(1)
#         else:
#             predictions.append(0)
#     print("devset time took:", time.process_time() - start)
#     print("method took:", time.process_time() - begin)
#     return predictions

In [50]:
def compute_accuracies(predictedLabels, dev_set, dev_labels):
    yhats = predictedLabels
    accuracy = np.mean(yhats == dev_labels)
    tp = np.sum([yhats[i] == dev_labels[i] and yhats[i] == 1 for i in range(len(yhats))])
    precision = tp / np.sum([yhats[i] == 1 for i in range(len(yhats))])
    recall = tp / (np.sum([yhats[i] != dev_labels[i] and yhats[i] == 0 for i in range(len(yhats))]) + tp)
    f1 = 2 * (precision * recall) / (precision + recall)
    return accuracy, f1, precision, recall

In [51]:
def main(dataset, stemming, lowerCase, laplace, posPrior):
    trainSet, trainLabels, revSet, revLabels = loadDataset(dataset, stemming, lowerCase)
    predictedLabels = naiveBayes(trainSet, trainLabels, revSet, laplace, posPrior)

    accuracy, f1, precision, recall = compute_accuracies(predictedLabels, revSet, revLabels)
    print("Accuracy:",accuracy)
    print("F1-Score:",f1)
    print("Precision:",precision)
    print("Recall:",recall)

In [52]:
if __name__ == "__main__":
    dataset = "../TermProject/txt_sentoken"
    stemming = []
    lowerCase = True
    laplace = 1.0
    posPrior = 0.8
    print("RUN NUMBER 1 ---------------")
    main(dataset, stemming, lowerCase, laplace, posPrior)
#     print("RUN NUMBER 2---------------")
#     main(dataset, stemming, lowerCase, laplace, posPrior)
#     print("RUN NUMBER 3---------------")
#     main(dataset, stemming, lowerCase, laplace, posPrior)
#     print("RUN NUMBER 4---------------")
#     main(dataset, stemming, lowerCase, laplace, posPrior)
#     print("RUN NUMBER 5---------------")
#     main(dataset, stemming, lowerCase, laplace, posPrior)

RUN NUMBER 1 ---------------


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1793.70it/s]


creating occurences and wordlist
review count is:  1600
posV 27403
negV 25443
total word count is: 561696
Going through train set took:  2.515625
calculate prob
Prob calculations:  0.046875
devset time took: 0.09375
Accuracy: 0.7525
F1-Score: 0.7555555555555555
Precision: 0.7463414634146341
Recall: 0.765
